# Question 11 - The wrong way using theory

In [2]:
alpha = 4/3
mean = 10
theta = mean / alpha  
p = 1/10
lam = 9
mean_minutes = mean / 60


nu = 5/60 + mean_minutes * p

tausq = p*(1 - p)  * (alpha * theta**2)/(60**2) + p*(1-p) * mean_minutes**2 + (alpha * theta**2)/(60**2) * (1 / 100)

# Check tausq lines up with the theoretical value
print(tausq)
rho = lam * nu
L = (rho + (lam**2 * tausq + rho**2) / (2 * (1 - rho)))
print(L)
print(L/27)

0.004583333333333333
6.806249999999995
0.25208333333333316


# Question 11 + 12  simulation

In [3]:
from scipy.stats import gamma
import numpy as np

alpha = 4/3
mean = 10
beta = alpha / mean
beta = beta

arrival_rate = 9
service_k = 5

n = 50000
M = 400
means_multimorbid = np.zeros(M)
means_patients = np.zeros(M)
total_customers_m = np.zeros(M)
total_customers_mm_m = np.zeros(M)
var_multimorbid = np.zeros(M)
for m in range(M):
    in_queue = 0
    time = 0
    max_time = 500 # Hours
    step_time = 0.001
    times_for_service_completion = []
    service_times_store = []
    multimorbid_served = 0
    customers_served = 0


    queue_at_time = []
    customers_in_queue = []

    multimorbid_in_queue = 0
    patients_in_queue = 0

    multimorbid_in_queue_at_time = []
    patients_in_queue_at_time = []

    p_delay = 1/10
    p_multimorbid = 1/27

    while time < max_time:
        time += step_time
        # Check if a customer arrives
        p_arrival = arrival_rate * step_time 
        if np.random.rand() < p_arrival:
            customers_served += 1
            # Patient arrives
            # Determine type of patient
            if np.random.rand() < p_multimorbid:
                customers_in_queue.append(1)
                multimorbid_served += 1
            else:
                customers_in_queue.append(0)
            # Determine the service time
            if np.random.rand() < p_delay:
                service_time = service_k / 60 + gamma.rvs(a = alpha, scale = 1/beta)/60
            else:
                service_time = service_k / 60
            times_for_service_completion.append(time + service_time)
            service_times_store.append(service_time)

        # Check if a customer is served
        while len(times_for_service_completion) > 0 and times_for_service_completion[0] <= time:
            times_for_service_completion.pop(0)
            customers_in_queue.pop(0)
        
        # Update the number of customers in the queue
        total_customers = len(customers_in_queue)
        multimorbid_customers = sum(customers_in_queue)

        multimorbid_in_queue_at_time.append(multimorbid_customers)
        patients_in_queue_at_time.append(total_customers)

    means_multimorbid[m] = np.mean(multimorbid_in_queue_at_time[20000:])
    means_patients[m] = np.mean(patients_in_queue_at_time[20000:])
    total_customers_m[m] = customers_served
    total_customers_mm_m[m] = multimorbid_served
    var_multimorbid[m] = np.var(multimorbid_in_queue_at_time[20000:])

print("Mean number of multimorbid patients in the queue")
print(f'CI: {np.quantile(means_multimorbid, [0.025, 0.975])}, mean: {np.mean(means_multimorbid)}')
print("Mean number of patients in the queue")
print(f'CI: {np.quantile(means_patients, [0.025, 0.975])}, mean: {np.mean(means_patients)}')
print("Theoretical mean number of patients in the queue")
print(L)
print("Theoretical mean number of multimorbid patients in the queue")
print(L * np.mean(total_customers_mm_m) / np.mean(total_customers_m))
print("Arrival rate")
print(np.mean(total_customers_m) / max_time)
print("Arrival rate of multimorbid patients")
print(np.mean(total_customers_mm_m) / max_time)

print("Variance of multimorbid in queue")
print(f'CI: {np.quantile(var_multimorbid, [0.025, 0.975])}, mean: {np.mean(var_multimorbid)}')

Mean number of multimorbid patients in the queue
CI: [0.03339858 0.04975547], mean: 0.04034909302272287
Mean number of patients in the queue
CI: [1.02791963 1.14647339], mean: 1.083429597021673
Theoretical mean number of patients in the queue
6.806249999999995
Theoretical mean number of multimorbid patients in the queue
0.2527793993607385
Arrival rate
9.010399999999999
Arrival rate of multimorbid patients
0.33464
Variance of multimorbid in queue
CI: [0.03339493 0.05409755], mean: 0.04147458415061953
